In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse
from gensim import corpora, models, matutils
from  itertools import chain
from sklearn.feature_extraction import text 

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#with open('/content/drive/My Drive/bgg_df.pickle', 'rb') as f:
#    df = pickle.load(f)

In [2]:
with open('bgg_df.pickle', 'rb') as f:
    df = pickle.load(f)

In [3]:
for idx, words in enumerate(df['comments']):
    combined = ""
    for item in words:
        combined += item
    df.at[idx,'newcomments'] = combined

In [4]:
my_additional_stop_words = ['game','games','play','players','played','player','plays','playing','just','great','like','good', 've','que','en','para','se','ks','le','el','es','really','la','com','don','http','https','www','com', \
                            '01','02','03','04','05','06','07','08','09','100','13','14','16','17','18','19','2015','2016','10','11','12','15','20','2017','2018','2019','2020','30']
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

In [6]:
cv = CountVectorizer(stop_words=stop_words, strip_accents='ascii', max_df=.7,min_df=.04)
data_cv = cv.fit_transform(df.newcomments)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = df.index
data_dtm

,ability,able,absolutely,abstract,action,actions,actual,actually,add,added,...,work,works,world,worth,wouldn,written,wrong,year,years,yes
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100084,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
100086,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
corpus = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtm.transpose()))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())
word2id = dict((k, v) for k, v in cv.vocabulary_.items())
d = corpora.Dictionary()
d.id2token = id2word
d.token2id = word2id
lda = models.ldamulticore.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=11, passes=100,random_state=88)
lda.print_topics()

[(0,
  '0.058*"abstract" + 0.049*"pieces" + 0.048*"board" + 0.036*"pnp" + 0.029*"print" + 0.027*"interesting" + 0.023*"strategy" + 0.019*"opponent" + 0.018*"rules" + 0.016*"solitaire"'),
 (1,
  '0.069*"fun" + 0.041*"simple" + 0.039*"little" + 0.037*"quick" + 0.033*"light" + 0.031*"filler" + 0.029*"nice" + 0.028*"easy" + 0.028*"luck" + 0.023*"fast"'),
 (2,
  '0.293*"dice" + 0.119*"roll" + 0.068*"die" + 0.053*"luck" + 0.052*"rolling" + 0.019*"fun" + 0.013*"decisions" + 0.011*"use" + 0.010*"random" + 0.009*"number"'),
 (3,
  '0.089*"fun" + 0.032*"people" + 0.029*"party" + 0.027*"time" + 0.022*"group" + 0.015*"right" + 0.014*"best" + 0.013*"lot" + 0.012*"think" + 0.012*"friends"'),
 (4,
  '0.021*"theme" + 0.020*"rules" + 0.019*"time" + 0.015*"fun" + 0.014*"long" + 0.014*"bit" + 0.013*"interesting" + 0.012*"think" + 0.012*"lot" + 0.011*"make"'),
 (5,
  '0.036*"kickstarter" + 0.027*"expansion" + 0.027*"love" + 0.025*"solo" + 0.019*"art" + 0.019*"fun" + 0.016*"theme" + 0.016*"new" + 0.015*"ea